# Подготовка датасетов

## 1. Загружаем в датафреймы, лемматизируем и токенизируем

In [1]:
import numpy as np
import pandas
import pymorphy2
from pymorphy2.tokenizers import simple_word_tokenize
morph = pymorphy2.MorphAnalyzer()

In [2]:
path = "" # путь до директории, в которой лежат ноутбуки и папки data, preprocessed и results

In [3]:
# посмотреть на слова в датафрейме и на их набор значений
def analyze_data(df):
    words_dict = dict()

    for target_word in df["word"]:
        if target_word not in words_dict:
            words_dict.setdefault(target_word, set())

    for target_word in words_dict:
        for sense_id in df[df.word == target_word]["gold_sense_id"]:
            words_dict[target_word].add(sense_id)

    # # преобразуем set'ы в листы, чтобы был определнённый порядок, и сортируем по возрастанию
    for target_word in words_dict:
        words_dict[target_word] = sorted(list(words_dict[target_word]))

    # добавляем количество контекстов на каждое значение
    for target_word in words_dict:
        for i, sense in enumerate(words_dict[target_word]):
            words_dict[target_word][i] = (sense, len(df[df.word == target_word][df.gold_sense_id == sense]))

    return words_dict

In [4]:
def first_step_preprocessing(df_name):
    try:
        df = pandas.read_csv(path+"data\\main\\" + df_name + "\\train.csv", "\t")
    except FileNotFoundError:
        df = pandas.read_csv(path+"data\\additional\\" + df_name + "\\train.csv", "\t")
    del df["predict_sense_id"] # удаляем столбец для предсказывания значений - он нам не понадобится
    
    # проанализируем информацию
    df_info = analyze_data(df)
    
    # добавили пустой стоблец для лемматизированных контекстов
    df["lem_context"] = np.nan 
    
    # лемматизация
    for i in range(df.shape[0]): # будем перебирать контексты
        lemmatized_list = []
        for token in simple_word_tokenize(df['context'][i]):
            lemmatized_list.append(morph.parse(token)[0].normal_form)
        df["lem_context"][i] = " ".join(lemmatized_list)
    
    # на будущее: добавили столбец, указывающий на датасет-источник
    df["source"] = pandas.Series([df_name]*df.shape[0])
    
    df.to_csv(path + "preprocessed\\" + df_name + ".csv", "\t", encoding="utf-8", index=False)
    
    return df, df_info

In [5]:
active_dict, active_dict_info = first_step_preprocessing("active-dict")
print(active_dict.shape)
active_dict.head()

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\boss\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


(2073, 7)


,context_id,word,gold_sense_id,positions,context,lem_context,source
0,1,дар,1,18-22,Отвергнуть щедрый дар,отвергнуть щедрый дар,active-dict
1,2,дар,1,21-28,покупать преданность дарами и наградами,покупать преданность дар и награда,active-dict
2,3,дар,1,19-23,Вот яд – последний дар моей Изоры,вот яд – последний дар мой изора,active-dict
3,4,дар,1,81-87,Основная функция корильных песен – повеселить ...,основный функция корильный песня – повеселить ...,active-dict
4,5,дар,1,151-157,Но недели две спустя (Алевтина его когда-то об...,но неделя два спустя ( алевтин он когда-то о э...,active-dict


In [6]:
print(len(active_dict_info.keys()))
active_dict_info

85


{'дар': [('1', 8), ('2.1', 15), ('2.3', 13)],
 'двигатель': [('1', 6), ('2', 9)],
 'двойник': [('1.1', 7), ('1.2', 7), ('2', 6), ('3', 5)],
 'дворец': [('1', 7), ('2', 6)],
 'девятка': [('1.1', 5),
  ('1.2', 6),
  ('2', 7),
  ('3.1', 5),
  ('3.2', 5),
  ('3.3', 4),
  ('4.1', 6),
  ('4.2', 4),
  ('4.3', 5)],
 'дедушка': [('1', 5), ('2', 4)],
 'дежурная': [('1', 6), ('2', 6)],
 'дежурный': [('1', 6), ('2', 7)],
 'декабрист': [('1', 7), ('2', 4)],
 'декрет': [('1', 6), ('2', 6)],
 'дело': [('1.1', 7),
  ('1.2', 9),
  ('2.1', 9),
  ('2.2', 6),
  ('2.3', 5),
  ('3.1', 7),
  ('3.2', 7),
  ('4.1', 9),
  ('4.2', 8),
  ('4.3', 8),
  ('5.1', 11),
  ('5.2', 5),
  ('6.1', 10),
  ('6.2', 6),
  ('6.3', 8),
  ('7.1', 7),
  ('7.2', 8)],
 'демобилизация': [('1', 8), ('2', 6)],
 'демократ': [('1', 7), ('2.1', 6), ('3', 5)],
 'демонстрация': [('1', 7), ('2', 8), ('3.1', 9), ('3.2', 7), ('3.3', 7)],
 'дерево': [('1', 8), ('2', 9), ('3', 4)],
 'держава': [('1', 8), ('2', 7)],
 'дерзость': [('1.1', 7),
  ('

In [7]:
bts_rnc, bts_rnc_info = first_step_preprocessing('bts-rnc')
bts_rnc.shape

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(3491, 7)

In [8]:
print(len(bts_rnc_info.keys()))
bts_rnc_info

30


{'балка': [(1, 81), (2, 38)],
 'вид': [(1, 38), (2, 3), (5, 36)],
 'винт': [(1, 39), (2, 67), (3, 1), (5, 16)],
 'горн': [(1, 20), (2, 1), (3, 30)],
 'губа': [(1, 132), (3, 3), (4, 2)],
 'жаба': [(1, 79), (2, 6), (3, 9), (4, 27)],
 'клетка': [(1, 38), (2, 1), (3, 7), (4, 96), (5, 4), (6, 4)],
 'крыло': [(1, 51), (2, 19), (3, 1), (4, 4), (5, 1), (6, 3), (7, 5), (8, 7)],
 'купюра': [(1, 12), (2, 138)],
 'курица': [(1, 62), (2, 31)],
 'лавка': [(1, 67), (2, 82)],
 'лайка': [(1, 81), (2, 18)],
 'лев': [(1, 36), (2, 2), (3, 2), (4, 4)],
 'лира': [(1, 33), (2, 16)],
 'мина': [(1, 51), (2, 12), (3, 2)],
 'мишень': [(1, 88), (2, 33)],
 'обед': [(1, 62), (2, 17), (3, 19), (4, 2)],
 'оклад': [(1, 137), (2, 5), (3, 4)],
 'опушка': [(1, 142), (2, 6)],
 'полис': [(1, 3), (2, 139)],
 'пост': [(1, 32), (2, 98), (3, 10), (4, 2), (5, 2)],
 'поток': [(1, 12), (2, 124)],
 'проказа': [(1, 95), (2, 51)],
 'пропасть': [(1, 92), (2, 25), (3, 7), (4, 3)],
 'проспект': [(1, 117), (2, 6), (3, 4), (4, 12)],
 'пы

In [9]:
wiki_wiki, wiki_wiki_info = first_step_preprocessing('wiki-wiki')
wiki_wiki.shape

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(439, 7)

In [10]:
print(len(wiki_wiki_info.keys()))
wiki_wiki_info

4


{'замок': [(1, 100), (2, 38)],
 'лук': [(1, 65), (2, 45)],
 'суда': [(1, 100), (2, 35)],
 'бор': [(1, 14), (2, 42)]}

In [11]:
active_rnc, active_rnc_info = first_step_preprocessing('active-rnc')
active_rnc.shape

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(1828, 7)

In [12]:
print(len(active_rnc_info.keys()))
active_rnc_info

20


{'альбом': [(1, 41), (2, 25), (3, 19)],
 'анатомия': [(1, 71), (2, 22), (3, 6)],
 'базар': [(1, 91), (2, 6), (3, 3)],
 'балет': [(1, 51), (2, 10), (3, 25), (4, 10)],
 'беда': [(1, 69), (2, 27), (3, 3)],
 'бездна': [(1, 37), (2, 39), (3, 2), (4, 21)],
 'билет': [(1, 83), (2, 6), (3, 6), (4, 5)],
 'блок': [(1, 1), (2, 2), (3, 2), (4, 3), (5, 15), (7, 28), (8, 2)],
 'блоха': [(1, 86), (2, 2)],
 'брак': [(1, 95), (3, 2)],
 'бритва': [(1, 94), (2, 6)],
 'будущее': [(1, 64), (2, 22)],
 'вешалка': [(1, 77), (2, 4), (3, 12), (4, 1)],
 'вилка': [(1, 91), (2, 2), (3, 1)],
 'винт': [(1, 21), (2, 51), (3, 6), (4, 1)],
 'галерея': [(1, 16), (2, 12), (3, 6), (4, 8), (5, 4), (6, 50), (7, 3)],
 'горбуша': [(1, 43), (2, 31)],
 'горшок': [(1, 48), (2, 35), (3, 9)],
 'гроза': [(1, 82), (2, 4), (3, 8), (4, 4)],
 'группа': [(1, 13), (2, 3), (3, 13), (4, 47), (5, 20)]}

In [13]:
active_rutenten, active_rutenten_info = first_step_preprocessing('active-rutenten')
active_rutenten.shape

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(3671, 7)

In [14]:
print(len(active_rutenten_info.keys()))
active_rutenten_info

20


{'альбом': [(1, 6), (2, 81), (3, 363)],
 'анатомия': [(1, 50), (2, 36), (3, 9)],
 'базар': [(1, 75), (2, 6), (3, 9)],
 'балет': [(1, 53), (2, 13), (3, 26), (4, 2)],
 'беда': [(1, 33), (2, 60)],
 'бездна': [(1, 30), (2, 28), (3, 5), (4, 24)],
 'билет': [(1, 405), (2, 24), (3, 11), (4, 7)],
 'блок': [(1, 3), (2, 53), (3, 7), (4, 14), (5, 60), (6, 35), (7, 9), (9, 25)],
 'блоха': [(1, 84), (2, 2)],
 'брак': [(1, 83), (2, 13)],
 'бритва': [(1, 81), (2, 4)],
 'будущее': [(1, 54), (2, 29)],
 'вешалка': [(1, 248), (2, 32), (3, 100), (4, 9), (5, 1)],
 'вилка': [(1, 220), (2, 23), (3, 1), (4, 1), (5, 57)],
 'винт': [(1, 193), (2, 146), (3, 10), (4, 8), (5, 1)],
 'галерея': [(3, 1), (6, 17), (7, 6)],
 'горбуша': [(1, 51), (2, 42)],
 'горшок': [(1, 83), (2, 232), (3, 91)],
 'гроза': [(1, 76), (2, 4), (3, 3), (4, 12)],
 'группа': [(1, 5), (2, 11), (3, 17), (4, 42), (5, 16)]}

In [15]:
bts_rutenten, bts_rutenten_info = first_step_preprocessing('bts-rutenten')
bts_rutenten.shape

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(956, 7)

In [16]:
print(len(bts_rutenten_info.keys()))
bts_rutenten_info

11


{'дисциплина': [(1, 30), (2, 64), (3, 4)],
 'замок': [(1, 46), (2, 45)],
 'корона': [(1, 48), (2, 23), (3, 7)],
 'кран': [(1, 33), (2, 61)],
 'лавка': [(1, 25), (2, 65)],
 'летопись': [(1, 70), (2, 25)],
 'мат': [(1, 6), (2, 11), (4, 50)],
 'отклонение': [(1, 66), (2, 11), (3, 17)],
 'сук': [(1, 64)],
 'тост': [(1, 73), (2, 18)],
 'щель': [(1, 86), (3, 8)]}

## 2. Посмотрим, какие слова пересекаются между датасетами 

! По целевым словам active_rutenten и active_rnc совпадают, потом будем проверять, совпадают ли в них разделения на значения

In [17]:
print(set(active_rnc_info.keys()) & set(active_rutenten_info.keys()))

{'блок', 'брак', 'бритва', 'вешалка', 'альбом', 'горшок', 'базар', 'блоха', 'гроза', 'балет', 'галерея', 'винт', 'анатомия', 'билет', 'бездна', 'будущее', 'беда', 'горбуша', 'вилка', 'группа'}


active_dict по словам не пересекается ни с одним другим

In [18]:
print(set(active_dict_info.keys()) & set(active_rnc_info.keys()))
print(set(active_dict_info.keys()) & set(bts_rnc_info.keys()))
print(set(active_dict_info.keys()) & set(bts_rutenten_info.keys()))
print(set(active_dict_info.keys()) & set(wiki_wiki_info.keys()))

set()
set()
set()
set()


У active_rnc и bts_rnc (и, соответственно, active_rutenten) есть общее слово "винт"

In [19]:
print(set(active_rnc_info.keys()) & set(bts_rnc_info.keys()))
print(set(active_rnc_info.keys()) & set(bts_rutenten_info.keys()))
print(set(active_rnc_info.keys()) & set(wiki_wiki_info.keys()))

{'винт'}
set()
set()


У bts_rnc и bts_rutenten - "лавка"

In [20]:
print(set(bts_rnc_info.keys()) & set(bts_rutenten_info.keys()))
print(set(bts_rnc_info.keys()) & set(wiki_wiki_info.keys()))

{'лавка'}
set()


У bts_rutenten и wiki_wiki - "замок"

In [21]:
print(set(bts_rutenten_info.keys()) & set(wiki_wiki_info.keys()))

{'замок'}


#### Есть ли в датасетах подходящие нам слова 

In [22]:
def show_senses_with_enough_contexts(dict_with_info, what_is_enough=20):
    printed_out_something = False
    for key in dict_with_info:
        res = [] # list with good pairs (sense - number of contexts)
        for pair in dict_with_info[key]:
            if pair[1] >= what_is_enough:
                res.append(pair)
        if res:
            printed_out_something = True
            print(key)
            for pair in res:
                print("%s :: %s" % (pair[0], pair[1])) 
    if not printed_out_something:
        print("No senses with enough contexts here")
    return

In [23]:
# этот датасет дальше рассматривать не будем - там нет значений с > 20 контекстами
show_senses_with_enough_contexts(active_dict_info)

No senses with enough contexts here


In [24]:
show_senses_with_enough_contexts(bts_rnc_info)

балка
1 :: 81
2 :: 38
вид
1 :: 38
5 :: 36
винт
1 :: 39
2 :: 67
горн
1 :: 20
3 :: 30
губа
1 :: 132
жаба
1 :: 79
4 :: 27
клетка
1 :: 38
4 :: 96
крыло
1 :: 51
купюра
2 :: 138
курица
1 :: 62
2 :: 31
лавка
1 :: 67
2 :: 82
лайка
1 :: 81
лев
1 :: 36
лира
1 :: 33
мина
1 :: 51
мишень
1 :: 88
2 :: 33
обед
1 :: 62
оклад
1 :: 137
опушка
1 :: 142
полис
2 :: 139
пост
1 :: 32
2 :: 98
поток
2 :: 124
проказа
1 :: 95
2 :: 51
пропасть
1 :: 92
2 :: 25
проспект
1 :: 117
пытка
1 :: 116
2 :: 27
рысь
1 :: 82
2 :: 38
среда
2 :: 32
3 :: 47
4 :: 52
хвост
1 :: 100
штамп
1 :: 45
4 :: 47


In [25]:
show_senses_with_enough_contexts(wiki_wiki_info)

замок
1 :: 100
2 :: 38
лук
1 :: 65
2 :: 45
суда
1 :: 100
2 :: 35
бор
2 :: 42


In [26]:
show_senses_with_enough_contexts(active_rnc_info)

альбом
1 :: 41
2 :: 25
анатомия
1 :: 71
2 :: 22
базар
1 :: 91
балет
1 :: 51
3 :: 25
беда
1 :: 69
2 :: 27
бездна
1 :: 37
2 :: 39
4 :: 21
билет
1 :: 83
блок
7 :: 28
блоха
1 :: 86
брак
1 :: 95
бритва
1 :: 94
будущее
1 :: 64
2 :: 22
вешалка
1 :: 77
вилка
1 :: 91
винт
1 :: 21
2 :: 51
галерея
6 :: 50
горбуша
1 :: 43
2 :: 31
горшок
1 :: 48
2 :: 35
гроза
1 :: 82
группа
4 :: 47
5 :: 20


In [27]:
show_senses_with_enough_contexts(active_rutenten_info)

альбом
2 :: 81
3 :: 363
анатомия
1 :: 50
2 :: 36
базар
1 :: 75
балет
1 :: 53
3 :: 26
беда
1 :: 33
2 :: 60
бездна
1 :: 30
2 :: 28
4 :: 24
билет
1 :: 405
2 :: 24
блок
2 :: 53
5 :: 60
6 :: 35
9 :: 25
блоха
1 :: 84
брак
1 :: 83
бритва
1 :: 81
будущее
1 :: 54
2 :: 29
вешалка
1 :: 248
2 :: 32
3 :: 100
вилка
1 :: 220
2 :: 23
5 :: 57
винт
1 :: 193
2 :: 146
горбуша
1 :: 51
2 :: 42
горшок
1 :: 83
2 :: 232
3 :: 91
гроза
1 :: 76
группа
4 :: 42


In [28]:
show_senses_with_enough_contexts(bts_rutenten_info)

дисциплина
1 :: 30
2 :: 64
замок
1 :: 46
2 :: 45
корона
1 :: 48
2 :: 23
кран
1 :: 33
2 :: 61
лавка
1 :: 25
2 :: 65
летопись
1 :: 70
2 :: 25
мат
4 :: 50
отклонение
1 :: 66
сук
1 :: 64
тост
1 :: 73
щель
1 :: 86


#### Проанализируем подробнее слова, которые пересекаются в нескольких датасетах 

#### 1 

In [ ]:
# Замок: wiki-wiki и bts-rutenten
# хотя инвентарь значений у них как бы разный (по Большому Толковому Словарю vs по Викисловарю), для слова замок - одинаковые.
# и распределены они тоже одинаково: 1 значение - замок-здание, 2 значение - замок-ключ

In [29]:
print(wiki_wiki_info["замок"])
print(bts_rutenten_info["замок"])

[(1, 100), (2, 38)]
[(1, 46), (2, 45)]


In [30]:
# проверим, не перемекаются ли контексты
wiki_wiki_contexts = set(wiki_wiki[wiki_wiki.word == "замок"]["lem_context"])
bts_rutenten_contexts = set(bts_rutenten[bts_rutenten.word == "замок"]["lem_context"])
print(wiki_wiki_contexts & bts_rutenten_contexts)

set()


#### 2 

In [ ]:
# Лавка: bts-rnc и bts-rutenten
# инвентарь значений одинаковый
# 1 - лавка-скамейка, 2 - лавка-магазин

In [31]:
print(bts_rnc_info["лавка"])
print(bts_rutenten_info["лавка"])

[(1, 67), (2, 82)]
[(1, 25), (2, 65)]


In [32]:
bts_rnc_contexts = set(bts_rnc[bts_rnc.word == "лавка"]["lem_context"])
bts_rutenten_contexts = set(bts_rutenten[bts_rutenten.word == "лавка"]["lem_context"])
print(bts_rnc_contexts & bts_rutenten_contexts)

set()


#### 3 

In [ ]:
# Винт: bts-rnc, active-rnc, active-rutenten

In [33]:
print(bts_rnc_info["винт"])
print(active_rnc_info["винт"])
print(active_rutenten_info["винт"])

[(1, 39), (2, 67), (3, 1), (5, 16)]
[(1, 21), (2, 51), (3, 6), (4, 1)]
[(1, 193), (2, 146), (3, 10), (4, 8), (5, 1)]


Инвентарь значений:


**bts_rnc**

- 1 - винт-инструмент 
- 2 - винт как часть механизма (мотора, двигателя)
- 3 - препарат ("самая лучшая водка с лучшим винтом") 
- 5 - карточная игра


**active_rnc**

- 1 - инструмент
- 2 - часть механизма
- 3 - форма
- 4 - препарат ("содержащий метамфетамин препарат винт")


**active_rutenten**

- 1 - инструмент
- 2 - часть механизма
- 3 - форма (?пересекается со вторым значением в bts-rnc)
- 4 - препарат
- 5 - игра


Но в некоторых очень мало контекстов. Если убрать значения с количеством контекстов <20 (даже из всех трёх), то останется:

- 1 - винт-инструмент
- 2 - винт как часть механизма (мотора, двигателя)

In [34]:
# не перемекаются ли контексты bts-rnc, active-rnc, active-rutenten
bts_rnc_contexts = set(bts_rnc[bts_rnc.word == "винт"]["lem_context"])
active_rnc_contexts = set(active_rnc[active_rnc.word == "винт"]["lem_context"])
active_rutenten_contexts = set(active_rutenten[active_rutenten.word == "винт"]["lem_context"])
print(bts_rnc_contexts & active_rnc_contexts)
print(active_rnc_contexts & active_rutenten_contexts)
print(active_rnc_contexts & active_rutenten_contexts)

set()
set()
set()


#### 4 

In [ ]:
# Самое сложное - active-rnc и active-rutenten - 19 слов

In [35]:
# не пересекаются ли контексты
active_rnc_contexts = set(active_rnc["lem_context"])
active_rutenten_contexts = set(active_rutenten["lem_context"])

Анализировала значения в двух датасетах: совпадают ли они, какие контексты в каждом из значений.
Некоторые слова пришлось убрать - когда не очень понятно, совпадает ли разделение на значения или разделение не очень понятно.

В результате оставляем - базар, билет, блоха, брак, бритва, вешалка, вилка, галерея, горбуша, горшок, гроза (+ винт, но он уже). Убираем - альбом, анатомия, балет, беда, бездна, блок, будущее, группа.

In [36]:
print(active_rnc.shape)
active_rnc = active_rnc.drop(active_rnc[active_rnc.word=="альбом"].index)
active_rnc = active_rnc.drop(active_rnc[active_rnc.word=="анатомия"].index)
active_rnc = active_rnc.drop(active_rnc[active_rnc.word=="балет"].index)
active_rnc = active_rnc.drop(active_rnc[active_rnc.word=="беда"].index)
active_rnc = active_rnc.drop(active_rnc[active_rnc.word=="бездна"].index)
active_rnc = active_rnc.drop(active_rnc[active_rnc.word=="блок"].index)
active_rnc = active_rnc.drop(active_rnc[active_rnc.word=="будущее"].index)
active_rnc = active_rnc.drop(active_rnc[active_rnc.word=="группа"].index)
print(active_rnc.shape)

(1828, 7)
(1115, 7)


In [37]:
print(active_rutenten.shape)
active_rutenten = active_rutenten.drop(active_rutenten[active_rutenten.word=="альбом"].index)
active_rutenten = active_rutenten.drop(active_rutenten[active_rutenten.word=="анатомия"].index)
active_rutenten = active_rutenten.drop(active_rutenten[active_rutenten.word=="балет"].index)
active_rutenten = active_rutenten.drop(active_rutenten[active_rutenten.word=="беда"].index)
active_rutenten = active_rutenten.drop(active_rutenten[active_rutenten.word=="бездна"].index)
active_rutenten = active_rutenten.drop(active_rutenten[active_rutenten.word=="блок"].index)
active_rutenten = active_rutenten.drop(active_rutenten[active_rutenten.word=="будущее"].index)
active_rutenten = active_rutenten.drop(active_rutenten[active_rutenten.word=="группа"].index)
print(active_rutenten.shape)

(3671, 7)
(2472, 7)


In [38]:
two_datasets = pandas.concat((active_rnc, active_rutenten), axis=0)
two_datasets.shape

(3587, 7)

## 3. Соберём один общий датасет 

Сначала собираем датасет со всеми значениями отобранных слов (т.е. всех, кроме восьми убранных на прошлом шаге). 
Не добавляем active_dict, потому что там слова не пересекаются ни с одним из остальных, и нет значений с >= 20 контекстами.

In [39]:
all_senses = pandas.concat((bts_rnc, wiki_wiki, bts_rutenten, two_datasets), axis=0)
all_senses.shape

(8473, 7)

In [40]:
# сейчас при объединении будут проблемы с индексами, потому что они из разных датасетов и могут повторяться
# нужно записать в csv без индекса и снова считать
all_senses.to_csv(path + "preprocessed\\all_senses.csv", "\t", encoding="utf-8", index=False)
all_senses = pandas.read_csv(path + "preprocessed\\all_senses.csv", "\t", encoding="utf-8")

In [41]:
all_senses_info = analyze_data(all_senses)
print(len(all_senses_info.keys()))
all_senses_info

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


54


{'балка': [(1, 81), (2, 38)],
 'вид': [(1, 38), (2, 3), (5, 36)],
 'винт': [(1, 253), (2, 264), (3, 17), (4, 9), (5, 17)],
 'горн': [(1, 20), (2, 1), (3, 30)],
 'губа': [(1, 132), (3, 3), (4, 2)],
 'жаба': [(1, 79), (2, 6), (3, 9), (4, 27)],
 'клетка': [(1, 38), (2, 1), (3, 7), (4, 96), (5, 4), (6, 4)],
 'крыло': [(1, 51), (2, 19), (3, 1), (4, 4), (5, 1), (6, 3), (7, 5), (8, 7)],
 'купюра': [(1, 12), (2, 138)],
 'курица': [(1, 62), (2, 31)],
 'лавка': [(1, 92), (2, 147)],
 'лайка': [(1, 81), (2, 18)],
 'лев': [(1, 36), (2, 2), (3, 2), (4, 4)],
 'лира': [(1, 33), (2, 16)],
 'мина': [(1, 51), (2, 12), (3, 2)],
 'мишень': [(1, 88), (2, 33)],
 'обед': [(1, 62), (2, 17), (3, 19), (4, 2)],
 'оклад': [(1, 137), (2, 5), (3, 4)],
 'опушка': [(1, 142), (2, 6)],
 'полис': [(1, 3), (2, 139)],
 'пост': [(1, 32), (2, 98), (3, 10), (4, 2), (5, 2)],
 'поток': [(1, 12), (2, 124)],
 'проказа': [(1, 95), (2, 51)],
 'пропасть': [(1, 92), (2, 25), (3, 7), (4, 3)],
 'проспект': [(1, 117), (2, 6), (3, 4), (4

НО! Если только у одного значения контекстов нормальное количество, а у всех остальных их очень мало - такие слова нужно удалить. Потому что тогда мы возьмём это единственное значение, которое, во-первых, гораздо частотнее остальных, что неинтересно, и, во-вторых, такие случаи будут нам мешать при создании датасетов с ошибками (мы должны будем это одно и то же небольшое количество контекстов повторить очень много раз, чтобы достигнуть 40% ошибки по отношению к болшому значению).
Критерий: разница контекстов в пять раз (одного большого по сравнению со всеми остальными вместе).

Итак, что удаляем: 
* губа: 132 - 3 - 2
* купюра: 12 - 138
* лев: 36 - 2 - 2 - 4
* оклад: 137 - 5 - 4
* опушка: 142 - 6
* полис: 3 - 139
* поток: 12 - 124
* проспект: 117 - 6 - 4 - 12
* сук - только одно значение! 
* щель: 86 - 8 
* базар: 166 - 12 - 12
* билет: 488 - 30 - 17 - 12 (чтобы не удалять - просто обрежем первое значение - например, до двухсот, выбрав рандомные)
* блоха: 170 - 4
* брак: 178 - 13 - 2
* бритва: 175 - 10

In [42]:
all_senses_2 = all_senses
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "губа"].index)
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "купюра"].index)
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "лев"].index)
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "оклад"].index)
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "опушка"].index)
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "полис"].index)
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "поток"].index)
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "проспект"].index)
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "сук"].index)
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "щель"].index)
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "базар"].index)
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "блоха"].index)
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "брак"].index)
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "бритва"].index)
all_senses_2.shape

(6531, 7)

In [43]:
# теперь с "билетом": сначала удалим из нашего датасета все контексты, где у него значение 1
all_senses_2 = all_senses_2.drop(all_senses_2[all_senses_2.word == "билет"][all_senses_2.gold_sense_id==1].index)
print(all_senses_2.shape)

# выберем рандомные 200, которые добавим обратно
df_bilet_first_sense_contexts = all_senses[all_senses.word=="билет"][all_senses.gold_sense_id==1]
random_200 = df_bilet_first_sense_contexts.sample(200)
print(random_200.shape)

all_senses_2 = pandas.concat((all_senses_2, random_200))
print(all_senses_2.shape)

(6043, 7)
(200, 7)
(6243, 7)


C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [44]:
# снова исправляем спутанные индексы 
all_senses_2.to_csv(path + "preprocessed\\all_senses_2.csv", "\t", encoding="utf-8", index=False)
all_senses_2 = pandas.read_csv(path + "preprocessed\\all_senses_2.csv", "\t", encoding="utf-8")

In [45]:
all_senses_2_info = analyze_data(all_senses_2)
print(len(all_senses_2_info.keys()))
all_senses_2_info

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


40


{'балка': [(1, 81), (2, 38)],
 'вид': [(1, 38), (2, 3), (5, 36)],
 'винт': [(1, 253), (2, 264), (3, 17), (4, 9), (5, 17)],
 'горн': [(1, 20), (2, 1), (3, 30)],
 'жаба': [(1, 79), (2, 6), (3, 9), (4, 27)],
 'клетка': [(1, 38), (2, 1), (3, 7), (4, 96), (5, 4), (6, 4)],
 'крыло': [(1, 51), (2, 19), (3, 1), (4, 4), (5, 1), (6, 3), (7, 5), (8, 7)],
 'курица': [(1, 62), (2, 31)],
 'лавка': [(1, 92), (2, 147)],
 'лайка': [(1, 81), (2, 18)],
 'лира': [(1, 33), (2, 16)],
 'мина': [(1, 51), (2, 12), (3, 2)],
 'мишень': [(1, 88), (2, 33)],
 'обед': [(1, 62), (2, 17), (3, 19), (4, 2)],
 'пост': [(1, 32), (2, 98), (3, 10), (4, 2), (5, 2)],
 'проказа': [(1, 95), (2, 51)],
 'пропасть': [(1, 92), (2, 25), (3, 7), (4, 3)],
 'пытка': [(1, 116), (2, 27)],
 'рысь': [(1, 82), (2, 38)],
 'среда': [(1, 13), (2, 32), (3, 47), (4, 52)],
 'хвост': [(1, 100), (3, 8), (4, 12), (7, 1)],
 'штамп': [(1, 45), (2, 1), (3, 3), (4, 47)],
 'замок': [(1, 146), (2, 83)],
 'лук': [(1, 65), (2, 45)],
 'суда': [(1, 100), (2, 

#### Разметка значений 

Мы будем работать с толковым словарём Ефремовой, поэтому сейчас нужно соотнести значения слов из нашего датасета со значениями из словаря. Я это сделала, и по результатам датасет, с которым мы будем работать, нужно ещё немного изменить.

+ вешалка1, вешалка3 - у Ефремовой одно значение
+ вилка2 - у Ефремовой нет этого значения ("вилка цен")
+ корона2 - совмещённые разные значения (в ефремовой несколько и разных, тут одно)
+ летопись1, летопись2 - ни в RUSSE, ни у Ефремовой разделение на значение не очень понятное
+ лира1 - совмещённые разные значения
+ обед1 - совмещённые разные значения
+ пропасть1, пропасть2 - у Ефремовой одно значение

Тогда что мы делаем:
1. Убираем из датасета all_senses_2 целиком некоторые слова (летопись, лира, обед, пропасть, сук)
2. Потом, из датасета на основе all_senses_2, который будет включать только значения с контекстами >20, убираем определённые значения (вилка2, корона2) - не будем пытаться их атрибутировать.

In [46]:
all_senses_3 = all_senses_2
all_senses_3 = all_senses_3.drop(all_senses_3[all_senses_3.word == "летопись"].index)
all_senses_3 = all_senses_3.drop(all_senses_3[all_senses_3.word == "лира"].index)
all_senses_3 = all_senses_3.drop(all_senses_3[all_senses_3.word == "обед"].index)
all_senses_3 = all_senses_3.drop(all_senses_3[all_senses_3.word == "пропасть"].index)

In [47]:
all_senses_3.to_csv(path + "preprocessed\\all_senses_3.csv", "\t", encoding="utf-8", index=False)
all_senses_3 = pandas.read_csv(path + "preprocessed\\all_senses_3.csv", "\t", encoding="utf-8")

In [48]:
all_senses_3_info = analyze_data(all_senses_3)
print(len(all_senses_3_info.keys()))
all_senses_3_info

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


36


{'балка': [(1, 81), (2, 38)],
 'вид': [(1, 38), (2, 3), (5, 36)],
 'винт': [(1, 253), (2, 264), (3, 17), (4, 9), (5, 17)],
 'горн': [(1, 20), (2, 1), (3, 30)],
 'жаба': [(1, 79), (2, 6), (3, 9), (4, 27)],
 'клетка': [(1, 38), (2, 1), (3, 7), (4, 96), (5, 4), (6, 4)],
 'крыло': [(1, 51), (2, 19), (3, 1), (4, 4), (5, 1), (6, 3), (7, 5), (8, 7)],
 'курица': [(1, 62), (2, 31)],
 'лавка': [(1, 92), (2, 147)],
 'лайка': [(1, 81), (2, 18)],
 'мина': [(1, 51), (2, 12), (3, 2)],
 'мишень': [(1, 88), (2, 33)],
 'пост': [(1, 32), (2, 98), (3, 10), (4, 2), (5, 2)],
 'проказа': [(1, 95), (2, 51)],
 'пытка': [(1, 116), (2, 27)],
 'рысь': [(1, 82), (2, 38)],
 'среда': [(1, 13), (2, 32), (3, 47), (4, 52)],
 'хвост': [(1, 100), (3, 8), (4, 12), (7, 1)],
 'штамп': [(1, 45), (2, 1), (3, 3), (4, 47)],
 'замок': [(1, 146), (2, 83)],
 'лук': [(1, 65), (2, 45)],
 'суда': [(1, 100), (2, 35)],
 'бор': [(1, 14), (2, 42)],
 'дисциплина': [(1, 30), (2, 64), (3, 4)],
 'корона': [(1, 48), (2, 23), (3, 7)],
 'кран':

#### Составим датасет(ы), с которым будем работать 

Оставим только значения, для которых есть больше 20 контекстов

In [49]:
df_to_work_with = all_senses_3
for word in all_senses_3_info:
    for pair in all_senses_3_info[word]:
        if pair[1] < 20:
            df_to_work_with = df_to_work_with.drop(df_to_work_with[df_to_work_with.word == word]
                                                   [df_to_work_with.gold_sense_id == pair[0]].index)
df_to_work_with.shape

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(5448, 7)

И уберём те, которые решили убрать после разметки значений по Ефремовой: вилка#2 и корона#2.

In [50]:
df_to_work_with = df_to_work_with.drop(df_to_work_with[df_to_work_with.word == "вилка"]
                                       [df_to_work_with.gold_sense_id == 2].index)
df_to_work_with = df_to_work_with.drop(df_to_work_with[df_to_work_with.word == "корона"]
                                       [df_to_work_with.gold_sense_id == 2].index)
df_to_work_with.shape

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


(5400, 7)

In [51]:
df_to_work_with_info = analyze_data(df_to_work_with)
print(len(df_to_work_with_info.keys()))
df_to_work_with_info

C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


36


{'балка': [(1, 81), (2, 38)],
 'вид': [(1, 38), (5, 36)],
 'винт': [(1, 253), (2, 264)],
 'горн': [(1, 20), (3, 30)],
 'жаба': [(1, 79), (4, 27)],
 'клетка': [(1, 38), (4, 96)],
 'крыло': [(1, 51)],
 'курица': [(1, 62), (2, 31)],
 'лавка': [(1, 92), (2, 147)],
 'лайка': [(1, 81)],
 'мина': [(1, 51)],
 'мишень': [(1, 88), (2, 33)],
 'пост': [(1, 32), (2, 98)],
 'проказа': [(1, 95), (2, 51)],
 'пытка': [(1, 116), (2, 27)],
 'рысь': [(1, 82), (2, 38)],
 'среда': [(2, 32), (3, 47), (4, 52)],
 'хвост': [(1, 100)],
 'штамп': [(1, 45), (4, 47)],
 'замок': [(1, 146), (2, 83)],
 'лук': [(1, 65), (2, 45)],
 'суда': [(1, 100), (2, 35)],
 'бор': [(2, 42)],
 'дисциплина': [(1, 30), (2, 64)],
 'корона': [(1, 48)],
 'кран': [(1, 33), (2, 61)],
 'мат': [(4, 50)],
 'отклонение': [(1, 66)],
 'тост': [(1, 73)],
 'билет': [(1, 200), (2, 30)],
 'вешалка': [(1, 325), (2, 36), (3, 112)],
 'вилка': [(1, 311), (5, 57)],
 'галерея': [(6, 67)],
 'горбуша': [(1, 94), (2, 73)],
 'горшок': [(1, 131), (2, 267), (3, 

In [52]:
df_to_work_with.to_csv(path + "preprocessed\\df_to_work_with.csv", "\t", encoding="utf-8", index=False)
df_to_work_with = pandas.read_csv(path + "preprocessed\\df_to_work_with.csv", "\t", encoding="utf-8")

In [53]:
df_to_work_with.to_excel(path + "preprocessed\\df_to_work_with.xlsx", index=False)

А теперь создадим датасет, в котором оставим только слова, для которых сохранилось больше одного значения

In [54]:
df_with_2_or_more_senses = df_to_work_with
for word in df_to_work_with_info:
    if len(df_to_work_with_info[word]) < 2:
        df_with_2_or_more_senses = df_with_2_or_more_senses.drop(df_with_2_or_more_senses
                                                                 [df_with_2_or_more_senses.word == word].index)
df_with_2_or_more_senses.shape

(4613, 7)

In [55]:
df_with_2_or_more_senses_info = analyze_data(df_with_2_or_more_senses)
print(len(df_with_2_or_more_senses_info.keys()))
df_with_2_or_more_senses_info

25


C:\Users\boss\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


{'балка': [(1, 81), (2, 38)],
 'вид': [(1, 38), (5, 36)],
 'винт': [(1, 253), (2, 264)],
 'горн': [(1, 20), (3, 30)],
 'жаба': [(1, 79), (4, 27)],
 'клетка': [(1, 38), (4, 96)],
 'курица': [(1, 62), (2, 31)],
 'лавка': [(1, 92), (2, 147)],
 'мишень': [(1, 88), (2, 33)],
 'пост': [(1, 32), (2, 98)],
 'проказа': [(1, 95), (2, 51)],
 'пытка': [(1, 116), (2, 27)],
 'рысь': [(1, 82), (2, 38)],
 'среда': [(2, 32), (3, 47), (4, 52)],
 'штамп': [(1, 45), (4, 47)],
 'замок': [(1, 146), (2, 83)],
 'лук': [(1, 65), (2, 45)],
 'суда': [(1, 100), (2, 35)],
 'дисциплина': [(1, 30), (2, 64)],
 'кран': [(1, 33), (2, 61)],
 'билет': [(1, 200), (2, 30)],
 'вешалка': [(1, 325), (2, 36), (3, 112)],
 'вилка': [(1, 311), (5, 57)],
 'горбуша': [(1, 94), (2, 73)],
 'горшок': [(1, 131), (2, 267), (3, 100)]}

In [56]:
df_with_2_or_more_senses.to_csv(path + "preprocessed\\df_with_2_or_more_senses.csv", "\t", encoding="utf-8", index=False)
df_with_2_or_more_senses = pandas.read_csv(path + "preprocessed\\df_with_2_or_more_senses.csv", "\t", encoding="utf-8")

In [57]:
df_with_2_or_more_senses.to_excel(path + "preprocessed\\df_with_2_or_more_senses.xlsx", index=False)